# 自然语言处理

## 1 语言表示与语言模型
### 1.1 词嵌入(word embedding)
图像分类中使用one-hot编码表示不同的类，但在自然语言中，字典(字表)很大如果使用one-hot编码会造成很大的数据稀疏性，并且该编码无法表达单词的语义相似性。自然语言中存在**语义鸿沟**问题，如“麦克风”和“话筒”，无法从字面看出两个单词其实在表达同样的东西，所以引入了基于神经网络的分布式表示，词嵌入(word embedding)或词向量(word vectors)

通过一些训练文本，训练词向量模型，得到单词的较低维度的分布式表示，优点如下：
1. 词向量的夹角越小，表达的语义更加接近(一定程度上解决了语义鸿沟问题)
2. 高维向量中元素不是0或1的形式，数据更加稠密

注意：word2vec是谷歌在2013年提出的训练词向量并获取词嵌入的**工具**

In [2]:
import os
import torch
import numpy as np
from torch import nn, optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models

# 优先使用GPU
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')

In [3]:
# 词嵌入，将one-hot编码的单词变为高维度的向量表示
word_to_ix = {'hello':0, 'world':1, 'python':2, 'AI':3}
embeds = nn.Embedding(3, 5)     # Embedding(m,n) m:字典，所有单词数目 n:嵌入维度
test_idx = torch.tensor(word_to_ix['python'])   # 转换为tensor
print('1.测试单词的向量类型:', test_idx.type())
test_embed = embeds(test_idx)   # 进行词嵌入
print('2.词嵌入的结果:', test_embed, test_embed.shape)

1.测试单词的向量类型: torch.LongTensor
2.词嵌入的结果: tensor([ 1.5304, -0.2406,  0.1706,  0.1118,  0.4761], grad_fn=<EmbeddingBackward>) torch.Size([5])


小结:
- `nn.Embedding(m,n)`是将one-hot编码嵌入到相对较低的维度，用float型数据表示，通常词典长度远大于嵌入的维度，但嵌入的维度仍可以看做是高维度，该语句只是通过线性变换将稀疏的数据转换为稠密的低维数据，所以线性变换的权重对应的参数也要进行参数更新，即**词向量也要进行参数的更新**

### 1.2 语言建模 (Language Model)
(1) **N-Gram模型**

一句话$T$由$w_1,w_2,...w_n$组成，通过前面的词推断后面的词，用条件概率将词联系到一起，公式如下:

<center>$P(T)=P(w_1)P(w_2|w_1)...P(w_n|w_{n-1}w_{n-2}...w_2w_1)$</center>

存在问题：预测一个词需要前面所有的词来计算

解决方法：引入马尔科夫假设，某个单词只有前面几个词有关系，并不是前面的所有词，一般可以认为距离接近的词之间的联系比距离较远的词联系紧密，引入马尔科夫假设后的N-Gram模型有:
1. 一元模型(unigram model)：单词间是独立的概率
2. 二元模型(bigram model)：前一个个单词推断后一个单词
3. 三元模型(trigram model)：前两个单词推断后一个单词

(2) **词袋模型 (Continuous Bag-of-words model,CBOW)**

一句话中的第$n$个单词，可以由它前面几个和后面几个单词推断出来，CBOW是通过上下文来预测中间目标词的模型，该模型可以看作N-Gram模型的加强版

<img src="./image/cbow.jpg" width="50%" height="50%">

(3) **Skip-Gram model (SG)**

与CBOW相反，是通过中间词来预测上下文
<img src="./image/skip-gram.jpg" width="50%" height="50%">
注意：语言模型可以通过分层softmax(hierarchical softmax)和负采样(negative sampling)来优化

### (1) N-Gram

In [12]:
# 1.导入news数据，从China Daily上找的3篇
import re  # 导入正则匹配来切分字符串

def read_file(file_path):
    """file_path: 读入文本数据的文件路径
       vocabulary：返回训练文本的字典
       file_idx：返回训练文本对应的字典序号
    """
    assert os.path.exists(file_path), 'file is not exist!'
    with open(file_path) as file:
        file_str = file.read().strip()   # 读入为字符格式
        split_rule = r'[\s\.\,]+'        # 匹配至少一个空格、逗号或句号
        file_list = re.split(split_rule, file_str) # 按规则分割文本字符串->list          
        file_set = set(file_list)        # 单词去重
        # 创建字典 word:index
        vocabulary = {word:i for i, word in enumerate(file_set)}  
        file_idx = [vocabulary[word] for word in file_list]
#         print(vocabulary)
#         print(file_list)
#         print(file_idx)
        return vocabulary, file_idx

vocab, data = read_file('./data/news.txt')
idx_to_word = {idx:word for word, idx in vocab.items()}   # 创建index:word的字典
print('1.字典的长度:', len(vocab))            # 将所有的词都作为字典了
print('2.训练文本的长度:', len(data))
samples_str = ' '.join([idx_to_word[index] for index in data[:20]])
print('3.样本字符串:', samples_str)

1.字典的长度: 621
2.训练文本的长度: 1413
3.样本字符串: Jack Ma founder and chairman of Chinese e-commerce giant Alibaba will leave his position as company chairman on Sept 10


一下数据生成器程序参考:[stikbuf/Language_Modeling](https://github.com/stikbuf/Language_Modeling/blob/master/Keras_Character_Aware_Neural_Language_Models.ipynb)

In [13]:
# 2.创建N-Gram的数据集 x(n-i),...,x(n-2),x(n-1)=>x(n)，前i个单词推断当前单词
# 数据生成的思想，先将[0,1,...,len(datasets)-1]索引随机，选出batch_size个索引
# 通过该索引使用datasets[i:i+num_infer+1]读取所需数据，但注意索引溢出
def create_data(datasets, num_infer, batch_size):
    """datasets:数据集list形式
       num_infer:需要参考的词数量
       X:所需的推测数据 [batch, num_infer] numpy数组形式
       y:目标数据 [batch, 1]
    """
    while True:
        rnd_idx = list(range(len(datasets) - num_infer - 1)) # 保证索引不溢出
        np.random.shuffle(rnd_idx)    # 打乱顺序，随机取出一个batch的数据
        batch_start = 0               # 按batch_size大小读取数据的索引
        X, Y = [], []
        while batch_start + batch_size < len(rnd_idx):    # 保证索引不溢出
            # 取batch_size个索引
            batch_idx = rnd_idx[batch_start:batch_start+batch_size] 
            temp_data = np.array([datasets[i:i+num_infer+1] for i in batch_idx])
#             print(temp_data)

            X = temp_data[:, :-1]       # 除去最后一列的所有数据
            Y = temp_data[:, -1:]       # 最后一列数据 
            batch_start += batch_size   # 方便读取下一个batch索引
            yield X, Y
        
gen = create_data(data, 2, 3)

In [14]:
x, y = next(gen)
print('1.训练数据X:\n', x)
print('2.目标数据y:\n', y)

1.训练数据X:
 [[ 82 297]
 [351 328]
 [352   5]]
2.目标数据y:
 [[ 56]
 [345]
 [203]]


In [15]:
# 3.创建模型
class NgramModel(nn.Module):
    def __init__(self, vocab_size, context_size, n_dim, hidden_dim=128):
        super().__init__()
        self.n_word = vocab_size                            # 训练数据的字典大小
        self.context_size = context_size
        self.n_dim = n_dim
        # 嵌入后的维度 [batch, context_size] -> [batch, context_size, n_dim]
        self.embedding = nn.Embedding(self.n_word, n_dim)   # 嵌入维度为n_dim
        # get [batch, hidden_dim]
        self.linear1 = nn.Linear(context_size*n_dim, hidden_dim)
        # get [batch, self.n_word] 与字典索引对应
        self.linear2 = nn.Linear(hidden_dim, self.n_word)          # 输出字典的维度
    def forward(self, x):
#         print('x:', x.shape)
        embeds = self.embedding(x)
         # get [batch,context_size*n_dim]
#         print('embeds:', embeds.shape)
        embeds = embeds.view(-1, self.context_size*self.n_dim)
        out = self.linear1(embeds)        # get [batch, hidden_dim]
        out = F.relu(out)
        out = self.linear2(out)           # get [batch, n_word]
        log_prob = F.log_softmax(out, 1)  # 沿第dim=1轴计算
        return log_prob    

In [16]:
# 创建模型并进行测试
context = 2  # 参考前2各词
inputs = np.random.randint(0,len(vocab), [5,2])
inputs = torch.from_numpy(inputs).long()   # 必须将数据转换为LongTensor才行
print('1.输入的数据类型:', inputs.type())
NGram_model = NgramModel(len(vocab), context, 128, hidden_dim=256)
outputs = NGram_model(inputs)
print('2.模型输出:', outputs.shape, '\n', outputs)

1.输入的数据类型: torch.LongTensor
2.模型输出: torch.Size([5, 621]) 
 tensor([[-6.2034, -6.3276, -6.7680,  ..., -6.2702, -6.7856, -6.1416],
        [-6.3747, -6.7633, -6.2730,  ..., -6.7733, -6.7820, -6.3854],
        [-5.9291, -6.6725, -6.2418,  ..., -6.2424, -6.6727, -6.8264],
        [-6.3766, -6.2838, -6.1931,  ..., -6.5251, -6.5047, -6.6470],
        [-6.4934, -6.3022, -6.3352,  ..., -6.5213, -6.0210, -7.1104]],
       grad_fn=<LogSoftmaxBackward>)


In [17]:
# 更加模型参数可以看出embedding的参数也要训练
for name, m in NGram_model.named_parameters():
    print(name, m.shape)

embedding.weight torch.Size([621, 128])
linear1.weight torch.Size([256, 256])
linear1.bias torch.Size([256])
linear2.weight torch.Size([621, 256])
linear2.bias torch.Size([621])


In [9]:
# 4.定义损失函数，优化函数
criterion = nn.CrossEntropyLoss()  # 网络输出vocab维度，目标vocab维度，类似于分类
optimizer = optim.SGD(NGram_model.parameters(), lr=1e-2)#, weight_decay=1e-5)

In [10]:
# 5.训练网络
context = 2              # 参考的单词数目
epochs = 50000
batch_size = 32
def train(model, batch_size, epochs):
    model.to(device)     # 优先使用GPU
    gen = create_data(data, context, batch_size)  # 创建数据生成器
    train_loss = 0
    for epoch in range(epochs):
        x, y = next(gen)  # 获取训练数据
        x, y = map(lambda x:torch.LongTensor(x).to(device), [x, y])
        y = y.flatten()
        prediction = model(x)
#         print(prediction.shape)
#         print(y.shape)
        loss = criterion(prediction, y)
#         train_loss += loss.item()    # 叠加
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 2000 == 0:
            print('epoch {} loss {:.4f}'.format(epoch, loss.item()))

In [11]:
# 调用训练函数，训练网络
train(NGram_model, batch_size, epochs)

epoch 0 loss 6.5259
epoch 2000 loss 4.1270
epoch 4000 loss 2.4980
epoch 6000 loss 1.0840
epoch 8000 loss 0.3508
epoch 10000 loss 0.2623
epoch 12000 loss 0.1866
epoch 14000 loss 0.2305
epoch 16000 loss 0.1297
epoch 18000 loss 0.2986
epoch 20000 loss 0.2003
epoch 22000 loss 0.1110
epoch 24000 loss 0.1851
epoch 26000 loss 0.0806
epoch 28000 loss 0.1409
epoch 30000 loss 0.1640
epoch 32000 loss 0.1360
epoch 34000 loss 0.0835
epoch 36000 loss 0.1297
epoch 38000 loss 0.3887
epoch 40000 loss 0.2297
epoch 42000 loss 0.0859
epoch 44000 loss 0.2147
epoch 46000 loss 0.0837
epoch 48000 loss 0.0971


In [16]:
gen = create_data(data, context, batch_size=1)
idx_to_word = {idx:word for word, idx in vocab.items()}   # 创建index:word的字典

def test(num_samples=1):
    print('-'*60)
    for i in range(num_samples):
        word, label = next(gen)
        # print(word)
        # print(label)
        # [[220 483]] numpy数组数据
        inputs_idx = [word[0,i] for i in range(word.shape[1])] 
        # print(inputs_idx)

        print('1.测试的输入单词 {}'.format([idx_to_word[idx] for idx in inputs_idx]), end='')
        print('  输出单词 {}'.format([idx_to_word[label[0,0]]]))

        word = torch.from_numpy(word).long().to(device)
        prediction = NGram_model(word)
        pred_label_idx = prediction.max(1)[1].item()
        # print(pred_label_idx)
        print('2.预测的单词为 ', idx_to_word[pred_label_idx])
        print('-'*60)
test(3)

------------------------------------------------------------
1.测试的输入单词 ['easier', 'and']  输出单词 ['more']
2.预测的单词为  more
------------------------------------------------------------
1.测试的输入单词 ['petrochemical', 'factory']  输出单词 ['with']
2.预测的单词为  with
------------------------------------------------------------
1.测试的输入单词 ['Tesla', 'announced']  输出单词 ['it']
2.预测的单词为  it
------------------------------------------------------------


In [17]:
num_np = np.array([[1,2],[3,4]], dtype=np.float64)
print(num_np, type(num_np))
num_torch1 = torch.LongTensor(num_np)   # 可以直接控制转换的类型
print(num_torch1, num_torch1.type())
num_torch2 = torch.from_numpy(num_np)   # 转换的tensor类型由numpy数据类型决定
print(num_torch2, num_torch2.type())

[[1. 2.]
 [3. 4.]] <class 'numpy.ndarray'>
tensor([[1, 2],
        [3, 4]]) torch.LongTensor
tensor([[1., 2.],
        [3., 4.]], dtype=torch.float64) torch.DoubleTensor


小结:
- `CrossEntropyLoss()`在分类任务中标签非one-hot编码下使用，输出维度[batch, out_dim]，目标维度[batch]仅是一维的数组，且里面的元素属于[0,num_classes-1]之间
- `torch.Tensor(numpy.ndarray)`和`torch.from_numpy(numpy.ndarray)`都可以将numpy数据转换为tensor，但前者直接控制转换的类型，后者是根据numpy的类型而定
- 根据模型的参数，看出embedding层的权重$w$的维度为 **[vocab_size, embedding_dim]**，单词$x$的维度为 **[vocab_size,1]**，通过$x^Tw$可以得到[1,embedding_dim]大小的嵌入向量，或$w^Tx$得到[embedding_dim,1]大小的嵌入向量，注意其中权重$w$的参数**需要训练**
- 这里采用了传统的神经网络方法，如下图所示，**NgramModel模型的输出维度为 [batch_size, vocab_size]**，当字典较小时，可以输出字典大小的维度，取值最大的一维索引号可以作为输出单词的索引，进而得到单词，。但当字典很大时如果输出字典大小的维度，会增大计算消耗，所以有**分层softmax(Hierarchical Softmax)**和**负采样(Negative Sampling)**两种方法来处理该问题

<img src="./image/dnn_lm.jpg" width="60%" height="60%">
<center>使用词汇表大小输出的传统DNN</center>
    
- N-Gram模型的原理图如下：

参考：[word2vec(CBOW/Skip-gram)](https://www.cnblogs.com/Determined22/p/5804455.html)
<img src="./image/N-Gram.jpg" width="60%" height="60%">
<center>N-Gram模型的原理图</center>
其中$w_{t-i}, \ i=1,2,...,n-1$表示单词$w_t$的上下文单词context，这$n-1$个单词，每个单词都经过嵌入矩阵$C$得到一个$m$维度的嵌入向量，这些向量**首位相接**得到长度为$(n-1)*m$的向量$x$，然后经过权重$H_{h\times(n-1)m}$得到隐层(维度为$h$)，并使用ReLU激活，这对应着程序的linear1部分，之后再进过权重$U_{V\times h}$得到维度为$1\times V$的输出$z$，公式如下所示，$V$表示字典大小，这对应着linear2部分
$$z=U\cdot ReLU(H\cdot x+d)+b \tag{1}$$
$$\hat y_t=P(w_t \mid w_{t-n-1},w_{t-n-2},\cdots, w_{t-1})=softmax(z_t)=\frac{e^{z_t}}{\sum_{k=1}^V e^{z_k}} \tag{2}$$
因为输出$z$为$1\times V$的，所以对它使用softmax可以找到$V$个数中最大的一个，即预测词的索引号，**$t$表示目标词的序号**，$\hat y_t$表示预测词表中第$t$个词$w_t$的概率，该值越大说明预测的越准确，优化的目的使它变大，损失函数优化方向是变小，所以对该值取负对数即可，目标词$w_t$对应的损失如下：
$$L=-\log \hat y_t \tag{3}$$
模型的损失是将所有中心词的损失求和即可，设$D$表示整个语料，模型的损失函数如下：
$$L=-\sum_{w_{t-(n-1)}^t \in D} logsoftmax(z_t) \tag{4}$$

### (2) CBOW

In [111]:
# 1. 定义生成器产生context词 [w(n-2),w(n-1), w(n+1), w(n+2)]与中心目标词 w(n)
# 已知 字典vocab，字典键值对互换idx_to_word，训练数据data
def gen_cbow(datasets, batch_size, context):
    """datasets:读入的数据集
       batch_size:每次生成的批次
       context:中心词对应的上下文(单边的)，cbow使用两边的2*context来作为上下文
    """
    while True:
        # 列表切片[a,b]中b无法取，所以len(datasets)-context-1才行
        rnd_idx = list(range(context, len(datasets) - context - 1)) # 保证索引不溢出
        np.random.shuffle(rnd_idx)    # 打乱顺序，随机取出一个batch的数据
        batch_start = 0               # 按batch_size大小读取数据的索引
        X, Y = [], []
        while batch_start + batch_size < len(rnd_idx):    # 保证索引不溢出
            # 取batch_size个索引
            batch_idx = rnd_idx[batch_start:batch_start+batch_size] 
            temp_data = np.array([datasets[i-context:i+context+1] for i in batch_idx])
#             print(temp_data)
            X = np.delete(temp_data, context, axis=1)  # 删除context列
            Y = temp_data[:, context:context+1]       # 获取context列数据 
            batch_start += batch_size   # 方便读取下一个batch索引
            yield X, Y


In [112]:
gen = gen_cbow(data, batch_size=3, context=2)
x,y = next(gen)
print('1.生成的上下文词one-hot索引:\n', x)
print('2.生成的中心词one-hot索引:\n',y)

1.生成的上下文词one-hot索引:
 [[392 140 511 328]
 [174 292 166 510]
 [275 127 294 117]]
2.生成的中心词one-hot索引:
 [[461]
 [ 58]
 [ 31]]


In [139]:
# 2.定义模型
class CBOW(nn.Module):
    def __init__(self, n_word, n_dim, context, hidden_dim=128):
        """n_word: 字典大小
           n_dim：词向量嵌入的维度
           context：单侧上下文的大小
           hidden_dim：全连接层维度大小       
        """
        super().__init__()
        self.context = context
        self.n_dim = n_dim
        # 将词表的维度嵌入n_dim内
        self.embedding = nn.Embedding(n_word, n_dim)
        self.linear1 = nn.Linear(2*context*n_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, n_word)

    def forward(self, x):
        x = self.embedding(x)                      # get [batch,2*context, n_dim]
        # view是将context个n_dim向量首位相连
        x = x.view(-1, 2*self.context*self.n_dim)  # get [batch,2*context*n_dim]
        x = self.linear1(x)                        # get [batch,hidden_dim]
        x = F.relu(x, inplace=True)
        x = self.linear2(x)                        # get [batch,n_word]
        x = F.log_softmax(x, dim=1)                # 按列取softmax
        return x

In [140]:
# 创建模型并测试
test_context = 2  # 参考前2各词
inputs = np.random.randint(0,len(vocab), [5,2*test_context])
inputs = torch.from_numpy(inputs).long()   # 必须将数据转换为LongTensor才行
print('1.输入的数据类型:', inputs.type())
test_model = CBOW(len(vocab), 128, test_context, hidden_dim=256)
outputs = test_model(inputs)
print('2.模型输出:', outputs.shape, '\n', outputs)

1.输入的数据类型: torch.LongTensor
2.模型输出: torch.Size([5, 621]) 
 tensor([[-6.7708, -6.3647, -6.6983,  ..., -6.8206, -6.5540, -6.5335],
        [-6.7181, -6.6434, -6.7686,  ..., -6.7863, -6.4540, -6.2191],
        [-6.7444, -6.7026, -6.7049,  ..., -6.6724, -6.4301, -6.6142],
        [-6.6715, -6.3363, -6.6662,  ..., -6.5572, -6.4603, -6.4034],
        [-6.6054, -6.3410, -6.5611,  ..., -6.5424, -6.6727, -6.6367]],
       grad_fn=<LogSoftmaxBackward>)


In [141]:
cbow_model = CBOW(len(vocab), 100, context, hidden_dim=256)

In [142]:
# 3.定义损失函数及优化方法
context = 5              # 参考的单词数目
batches = 30000          # 训练的总批次数
batch_size = 32
batches_per_epoch = np.ceil(len(data)/batch_size)
max_epochs = np.ceil(batches/batches_per_epoch)
print('batches/epoch: {}  max_epochs: {}'.format(batches_per_epoch, max_epochs))

criterion = nn.CrossEntropyLoss()
cbow_model = CBOW(len(vocab), 100, context, hidden_dim=256)
# parameters = nn.utils.clip_grad_norm_(cbow_model.parameters(), max_norm=1)
optimizer = optim.SGD(cbow_model.parameters(), lr=1e-2)

batches/epoch: 45.0  max_epochs: 667.0


In [143]:
# 4.训练网络
def train(dataloader, model, batch_size, batches):
    """dataloader：生成批次数据的生成器
       model：需要训练的模型
       batch_size：批次大小
       epochs：训练数据的轮数    
    """
    model.to(device)     # 优先使用GPU
    train_loss = 0
    for epoch in range(batches):
        x, y = next(dataloader)  # 获取训练数据
        x, y = map(lambda x:torch.LongTensor(x).to(device), [x, y])
        y = y.flatten()          # 把y变为一维数组
        prediction = model(x)
#         print(prediction.shape)
#         print(y.shape)
#         print(y)
#         print(x.shape)
#         print(y.shape)
        loss = criterion(prediction, y)
#         train_loss += loss.item()    # 叠加
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if epoch % 2000 == 0 or epoch == batches-1:
            print('epoch {} loss {:.4f}'.format(epoch, loss.item()))

data_gen = gen_cbow(data, batch_size=batch_size, context=context) # 创建数据生成器
# cbow_model = CBOW(len(vocab), 256, context, hidden_dim=512)  # 创建模型
train(data_gen, cbow_model, batch_size, batches)

epoch 0 loss 6.4371
epoch 2000 loss 1.7228
epoch 4000 loss 0.2342
epoch 6000 loss 0.0634
epoch 8000 loss 0.0270
epoch 10000 loss 0.0207
epoch 12000 loss 0.0163
epoch 14000 loss 0.0110
epoch 16000 loss 0.0098
epoch 18000 loss 0.0092
epoch 20000 loss 0.0067
epoch 22000 loss 0.0063
epoch 24000 loss 0.0057
epoch 26000 loss 0.0054
epoch 28000 loss 0.0044
epoch 29999 loss 0.0045


In [152]:
# 5.测试模型，这里使用数据生成器
def test(dataloader, model, context, num_samples=1):
    """dataloader：生成批次数据的生成器
       model：需要测试的模型
       context：单侧的上下文单词数
       num_samples：测试的样本数
    """
    print('-'*60)
    for i in range(num_samples):
        word, label = next(dataloader)
#         print(word)
#         print(label)
        inputs_idx = [word[0,i] for i in range(word.shape[1]) ] 
#         print(inputs_idx)

        print('1.上下文: {}'.format([idx_to_word[idx] for idx in inputs_idx]))
        print('  中心词: {}'.format([idx_to_word[label[0,0]]]))

        word = torch.from_numpy(word).long().to(device)
        model.to(device)
        prediction = model(word)
#         print(prediction.shape)
        pred_label_idx = prediction.max(1)[1].item()
#         print(pred_label_idx)
        print('2.预测的单词为 ', idx_to_word[pred_label_idx])
        print('-'*60)

data_gen = gen_cbow(data, batch_size=1, context=context)
test(data_gen, cbow_model, context, 3)

------------------------------------------------------------
1.上下文: ['recruiting', 'channels', 'should', 'be', 'broadened', 'of', 'Helsinki', 'capital', 'of', 'Finland']
  中心词: ['Mayor']
2.预测的单词为  Mayor
------------------------------------------------------------
1.上下文: ['includes', '115', 'kilometers', 'in', 'mainland', '26', 'kilometers', 'in', 'Hong', 'Kong']
  中心词: ['and']
2.预测的单词为  and
------------------------------------------------------------
1.上下文: ['park', 'waiting', 'for', 'outdoor', 'activities', 'whatever', 'it', 'could', 'happen', 'he']
  中心词: ['or']
2.预测的单词为  or
------------------------------------------------------------


小结：
- **先创建模型再定义优化器**，如果之后不小心又创建了一遍模型，才开始训练模型，则模型无法更新参数，原因是优化器是针对先创建的模型model1的，但传入训练时的模型是model2，两个模型是不同的，即使名称相同，但`id`查看后是不同的模型，在训练是model2进行前向计算，但优化器无法更新model2的参数，所以损失函数会保持不变
- 使用Adam优化的太快，设置很小的学习率才能缓慢的优化，而相同学习率下SGD优化很慢，证明Adam优化的能力更强
- 这里的CBOW和N-Gram没有本质的区别，只是N-Gram是选择中心词前面的$c$个context，而CBOW选择前后共$2c$个context做训练集，并且中心词都只有一个，模型中都使用了传统的神经网络，用全连接层输出字典大小维度，然后取softmax概率，最大概率的一维索引判定为预测词的索引，进而得到预测词
- pytorch中CrossEntropyLoss损失函数的公式如下：
$$\text{loss}(x, class) = -\log\left(\frac{\exp(x[class])}{\sum_j \exp(x[j])}\right)
                   = -x[class] + \log\left(\sum_j \exp(x[j])\right)$$
带权类别的公式如下：
$$\text{loss}(x, class) = weight[class] \left(-x[class] + \log\left(\sum_j \exp(x[j])\right)\right)$$

### (3) Skip-Gram

---
word2vec是谷歌提出的训练词向量的工具，它也使用到CBOW和Skip-Gram两种语言模型，但使用分层softmax和负采样来创建这两种模型，从而较少了计算消耗，并实现很好的性能，下面介绍着两种模型

### (2) CBOW
参考：[基于Hierarchical Softmax的模型概述](https://www.cnblogs.com/pinard/p/7243513.html)

(1) 使用Hierarchical Softmax的方法
与传统神经网络方法不同点：
1. 从输入层到隐藏层的映射没有采用神经网络的线性变换加激活函数的方法(N-Gram例子中使用该方法)，而是直接对输入的上下文词向量求**平均**得到一个词向量
2. 从隐藏层到输出的softmax层输出维度变化，DNN的方法中输出的维度是字典大小，而分层softmax使用**霍夫曼树**来作为隐藏层到输出softmax层的映射，如下图所示:

<img src="./image/huffman_tree.jpg" width="50%" height="50%">
<center>霍夫曼树表示输出映射</center>
    
隐藏层到输出层的softmax不是一下完成，而是沿着霍夫曼树进行，二叉树左子树为负类(用1表示)，右子树为正类(用0表示)，从根节点出发，在某个节点处对应的哪个子类概率大就沿着哪个子树走，直到走到叶节点。叶子节点的个数与词汇表对应，而内部节点处子类对应的概率与模型参数有关，所以通过训练可以优化参数，最后优化的对数似然函数$L$如下，具体细节参考上述网页
$$L=log\prod_{j=2}^{l_w}(P(d_j^w|x_w,\theta_{j-1}^w))=\sum_{j=2}^{l_w}((1-d_j^w)log[\sigma(x_w^T\theta_{j-1}^w)]+d_j^w log[1-\sigma(x_w^T\theta_{j-1}^w)]) $$
其中$P(d_j^w|x_w,\theta_{j-1}^w)$代表$w$经过霍夫曼树的某一个节点$j$的逻辑回归概率，表达式如下：
$$\begin{equation}
P(d_j^w|x_w,\theta_{j-1}^w)=\left\{
    \begin{array}{lr}
     \sigma(x_w^T\theta_{j-1}^w)   & d_j^w=0 \\
     1-\sigma(x_w^T\theta_{j-1}^w) & d_j^w=1
     \end{array}
\right.
\end{equation}$$
- 优化损失函数是为了求最小值，使用梯度下降法，在极大似然中为使似然概率最大使用梯度上升法


---

参考：[基于Negative Sampling的模型概述](http://www.cnblogs.com/pinard/p/7249903.html)

(2) 使用负采样Negative Sampling方法

在训练样本中，中心词$w$周围的上下文有$2c$个词，记为$context(w)$，这些词都是有相关联的，所以这$2c$个词是中心词的正例，然后进行**负采样**，随机选择$neg$个单词$w_i,i=1,2,...neg$作为负例。最终得到样本$(context(w),w_i)$，其中$i=1,2,...neg$，使用一个正例和$neg$个负例进行**二元逻辑回归**，训练模型更新参数

设$w_0$表示正例，则正、负例满足下面公式：

$$P(context(w_0),w_i)=\sigma(x_{w_0}^T\theta^{w_i}), \quad y_i=1,i=0 \tag{1}$$
$$P(context(w_0),w_i)=1-\sigma(x_{w_0}^T\theta^{w_i}),\quad  y_i=0,i=1,2,...,neg \tag{2}$$

## ？？？为什么不是$x_{w_i}^T$，为什么每个w都有权重，每个都这么算？

模型的目的是分类准确，所以正样本对应(1)的概率越大越好，而负样本对应(2)的概率越大越好，所以得到似然函数为:
$$\prod_{i=0}^{neg}\sigma(x_{w_0}^T\theta^{w_i})^{y_i}(1-\sigma(x_{w_0}^T\theta^{w_i}))^{1-y_i} \tag{3}$$
对其求对数得：
$$\sum_{i=0}^{neg} y_ilog(\sigma(x_{w_0}^T\theta^{w_i})) + (1-y_i)log((1-\sigma(x_{w_0}^T\theta^{w_i}))) \tag{4}$$

**注意：上面的$w_0$表示正样本，$x_{w_0}$代表正样本的词向量，是其上下文的$2c$个单词平均后得到的，$\theta^{w_i}$表示第$i$个负样本的词向量$w_i$对应的模型参数**，正负样本共$neg+1$个单词一起进行拟合优化

负采样方法：

将一条线段分成$V$份，每份对应词汇表的一个词，根据词频率的大小来决定线段的长度，每个词$w$的线段长度用下面的公式计算：
$$len(w)=\frac{count(w)}{\sum_{u\in vocab} count(u)} \tag{5}$$
在word2vec中分子和分母都取3/4次幂：
$$len(w)=\frac{count(w)^{3/4}}{\sum_{u\in vocab} count(u)^{3/4}} \tag{6}$$
采用前，将长度为1的线段分成$M$等份，使$M>>V$，使频率高的单词对应多个$1/M$等份，采用时从$M$个位置随机取出$neg$个位置，采样到的位置对应的线段所属的词就是负样本，$M$取值为$10^8$，示意图如下：

<img src="./image/negative_sampling.jpg" width="70%" height="70%">

In [ ]:
xTwi是不是应该写成xTw0？对于原始的w和负采样的w，context应该都是一样的计算方式，
只是在迭代过程中xTw0的值会随着词向量的更新而变化

['a', 'df', 'ff', ',', 'fff.', 'mdsfm,']